In [20]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

## 패키지 설치 및 Drive 연결

In [21]:
!pip install PyDrive
# pip install torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torchvision --upgrade --index-url https://download.pytorch.org/whl/cu118
# !pip install --upgrade torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python
!pip install opencv-python-headless
!pip install torchvision
!pip install google-api-core==2.11.0 google-api-python-client==2.79.0 google-auth==2.16.1 google-auth-httplib2==0.1.0 google-auth-oauthlib==0.4.1 googleapis-common-protos==1.58.0
!pip install --upgrade google-auth-oauthlib
!pip install requests google-auth-oauthlib google-auth-httplib2
# !pip install --force-reinstall sympy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.1%2Bcu118-cp310-cp310-linux_x86_64.whl (857.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp310-cp310-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached 

In [22]:
# YOLOv8 모델  설치
!pip3 install tqdm
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 라이브러리 불러오기
import os
import time
import pickle


In [5]:
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools

obj = lambda: None

# OAuth 2.0 설정을 위한 딕셔너리
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# OAuth 2.0 인증을 위한 설정
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secrets.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)


In [23]:
# Google Drive API를 사용하여 인증 관리
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# Google OAuth 인증 절차
gauth = GoogleAuth()


# 이미 저장된 자격 증명 파일이 있으면 로드
if os.path.exists("credentials.json"):
    gauth.LoadCredentialsFile("credentials.json")
    if gauth.access_token_expired:  # 토큰이 만료되었으면 갱신
        print("토큰 만료로 갱신")
        gauth.Refresh()
    else:  # 유효한 토큰이 있으면 인증 절차 건너뜀
        print("토큰이 있어 패스")
        gauth.Authorize()
else:
    # 자격 증명 파일이 없으면 새로 인증
    gauth.CommandLineAuth() # 이 과정을 통해 브라우저가 열리고, Google 계정 인증이 이루어짐
    gauth.SaveCredentialsFile("credentials.json")  # 자격 증명 파일 저장



토큰이 있어 패스


In [24]:
# Google Drive 객체 생성
drive = GoogleDrive(gauth)

### 폴더와 파일 이름 확인 코드 실행 시간: 1487.46초 

In [ ]:
# import time
# start_time = time.time()


# # 폴더 목록이 아닌 폴더 ID로 바로 접근
# folder_id = '10NTLRT0ooRHhIZdCS3Xim3VV3aY6GshL'

# # # 파일 목록 가져오기
# # folder_path = 'Colab Notebooks/SpecializedProject/content/자동차 차종-연식-번호판 인식용 영상'
# # folder_names = folder_path.strip('/').split('/')

# # 폴더와 파일 이름만 확인하는 코드!!!

# if folder_id:
#     # 폴더 내의 내용 가져오기
#     file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
#     for file in file_list:
#         print('title: %s, id: %s, mimeType: %s' % (file['title'], file['id'], file['mimeType']))

#     # 하위 폴더의 내용 재귀적으로 탐색하기
#     def list_folder_contents(drive, folder_id, indent=0):
#         file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
#         for file in file_list:
#             print('  ' * indent + 'title: %s, id: %s, mimeType: %s' % (file['title'], file['id'], file['mimeType']))
#             # 폴더이면 재귀적으로 탐색
#             if file['mimeType'] == 'application/vnd.google-apps.folder':
#                 list_folder_contents(drive, file['id'], indent + 1)

#     # 지정된 폴더의 모든 내용 출력
#     print("\n지정된 폴더의 모든 내용:")
#     list_folder_contents(drive, folder_id)
# else:
#     print("폴더 ID를 가져올 수 없습니다.")



# # 실행 종료 시간 기록
# end_time = time.time()
# # 실행 시간 계산 및 출력
# execution_time = end_time - start_time
# print(f"폴더와 파일 이름 확인 코드 실행 시간: {execution_time:.2f}초 ")

## 폴더의 id를 수동으로 설정 후, 각 폴더에서 파일을 불러온다!
### 딕셔너리에 저장하는 코드 실행 시간 : 843.71초
### 캐싱 + 딕셔너리에 저장하는 코드 실행 시간 : 848.07초
### 캐시 파일에서 데이터 불러오기 : 11.89초

In [ ]:
# start_time = time.time()

# # 라벨 폴더와 원천 폴더의 ID를 수동으로 설정
# label_folder_id = '1lY1svbrumIZADeTwogXIc3gq-HMfL_0W'  # [라벨] 폴더 ID
# source_folder_id = '1jYZlhZJW94-34-DKpALvsauuoKL4s_Yh'  # [원천] 폴더 ID


# if label_folder_id and source_folder_id:
#     print('라벨쪽 실행시작')
#     # [라벨] 폴더 내의 JSON 파일 목록 가져오기
#     label_files = drive.ListFile({'q': f"'{label_folder_id}' in parents and trashed=false"}).GetList()
#     print(f"라벨 파일 수: {len(label_files)}")  # 라벨 파일 개수 출력
    
#     # 라벨 파일 목록이 제대로 불러왔는지 확인
#     for file in label_files[:5]:  # 처음 5개 파일만 출력 (많은 경우를 대비)
#         print(f"라벨 파일: {file['title']}, ID: {file['id']}")
        
#     print('원천쪽 실행시작')
#     # [원천] 폴더 내의 이미지 파일 목록 가져오기
#     image_files = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()
#     print(f"이미지 파일 수: {len(image_files)}")  # 이미지 파일 개수 출력

#     # 이미지 파일 목록이 제대로 불러왔는지 확인
#     for file in image_files[:5]:  # 처음 5개 파일만 출력 (많은 경우를 대비)
#         print(f"이미지 파일: {file['title']}, ID: {file['id']}")
        
#     # 파일 ID와 제목을 딕셔너리로 저장
#     label_dict = {file['title']: file['id'] for file in label_files}
#     image_dict = {file['title']: file['id'] for file in image_files}

#     print("라벨 파일 목록 일부:", list(label_dict.items())[:5])
#     print("이미지 파일 목록 일부:", list(image_dict.items())[:5])

# else:
#     print("라벨 또는 원천 폴더의 ID를 찾을 수 없습니다.")
# # 실행 종료 시간 기록
# end_time = time.time()
# # 실행 시간 계산 및 출력
# execution_time = end_time - start_time
# print(f"코드 실행 시간: {execution_time:.2f}초")



In [26]:
# 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


start_time = time.time()

# 캐시 파일 경로 설정
label_cache_file = 'plate_label_files.pkl'
image_cache_file = 'plate_image_files.pkl'

# 라벨 폴더와 원천 폴더의 ID를 수동으로 설정
label_folder_id = '1lY1svbrumIZADeTwogXIc3gq-HMfL_0W'  # [라벨] 폴더 ID
source_folder_id = '1jYZlhZJW94-34-DKpALvsauuoKL4s_Yh'  # [원천] 폴더 ID

# 캐시 파일이 있는지 확인하고 불러오기
if os.path.exists(label_cache_file) and os.path.exists(image_cache_file):
    print('캐시 파일에서 데이터 불러오기...')
    with open(label_cache_file, 'rb') as f:
        label_files = pickle.load(f)
    with open(image_cache_file, 'rb') as f:
        image_files = pickle.load(f)
else:
    print('캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...')
    if label_folder_id and source_folder_id:
        print('라벨쪽 실행시작')
        # [라벨] 폴더 내의 JSON 파일 목록 가져오기
        label_files = drive.ListFile({'q': f"'{label_folder_id}' in parents and trashed=false"}).GetList()
        print(f"라벨 파일 수: {len(label_files)}")  # 라벨 파일 개수 출력

        # [원천] 폴더 내의 이미지 파일 목록 가져오기
        print('원천쪽 실행시작')
        image_files = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()
        print(f"이미지 파일 수: {len(image_files)}")  # 이미지 파일 개수 출력

        # 파일 목록을 캐시 파일에 저장
        with open(label_cache_file, 'wb') as f:
            pickle.dump(label_files, f)
        with open(image_cache_file, 'wb') as f:
            pickle.dump(image_files, f)
    else:
        print("라벨 또는 원천 폴더의 ID를 찾을 수 없습니다.")
        label_files = []
        image_files = []

# 캐시된 데이터에서 딕셔너리 생성
label_dict = {file['title']: file['id'] for file in label_files}
image_dict = {file['title']: file['id'] for file in image_files}

# 라벨 및 이미지 파일 목록 일부 출력
print("라벨 파일 목록 일부:", list(label_dict.items())[:5])
print("이미지 파일 목록 일부:", list(image_dict.items())[:5])

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


캐시 파일에서 데이터 불러오기...
라벨 파일 목록 일부: [('92라0979.json', '1foIJ3kuyu71y1BKPqFGng8sScpjCdP_4'), ('92도5577-60.json', '1fHtCmv9PRULg-NmhyprvcXML1k55o-aK'), ('92라0437.json', '1frVzgoW8BT_28Py_iDSw6EwWHkAoxIo_'), ('92마2666-2.json', '1gf72SG4QC2NGHQq_XLw0eBjAAbQPnAoI'), ('92도5577-6.json', '1fMZAobELiOLcPKmoozRZed261tlPl38G')]
이미지 파일 목록 일부: [('경기71아3609-5.jpg', '1erSY3aC27uRCByer8SWnOFfjiz9ii7Lh'), ('경기71아1446-6.jpg', '1xGm-7bE40X4iD82IPa26iQNtWo29myzO'), ('경기71아1444-7.jpg', '1w05HzSSLn2YPtR54nWqY2Bza-tvMYJGC'), ('경기37바6657-2.jpg', '13VT8Re67GTSuXWdPONjU7EDGk7V8TY9e'), ('경기37바6117-6.jpg', '1mNXJWmJ41wMweHFnG6xmswMz7RbnZCsl')]
코드 실행 시간: 11.89초


In [ ]:
# # [라벨] 폴더와 [원천] 폴더의 ID 가져오기
# label_folder_id = get_subfolder_id(drive, folder_id, 'Training/[라벨]자동차번호판OCR_train')
# source_folder_id = get_subfolder_id(drive, folder_id, 'Training/[원천]자동차번호판OCR데이터')

# print('라벨쪽 실행시작')
# # [라벨] 폴더 내의 JSON 파일 목록 가져오기
# label_files = drive.ListFile({'q': f"'{label_folder_id}' in parents and trashed=false"}).GetList()

# print('원천쪽 실행시작')
# # [원천] 폴더 내의 이미지 파일 목록 가져오기
# image_files = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()

# # 파일 ID와 제목을 딕셔너리로 저장
# label_dict = {file['title']: file['id'] for file in label_files}
# image_dict = {file['title']: file['id'] for file in image_files}

# # 특정 부모 폴더에서 하위 폴더 ID를 찾는 함수
# def get_subfolder_id(drive, parent_id, folder_title):
#     query = f"'{parent_id}' in parents and title = '{folder_title}' and mimeType = 'application/vnd.google-apps.folder' and trashed=false"
#     file_list = drive.ListFile({'q': query}).GetList()
    
#     if len(file_list) == 0:
#         print(f"폴더 '{folder_title}'을(를) 찾을 수 없습니다.")
#         return None
#     else:
#         return file_list[0]['id']
        
# # 'Training' 폴더의 ID 가져오기
# training_folder_id = get_subfolder_id(drive, folder_id, 'Training')

# if training_folder_id:
#     # [라벨] 폴더와 [원천] 폴더의 ID 가져오기
#     label_folder_id = get_subfolder_id(drive, training_folder_id, '[라벨]자동차번호판OCR_train')
#     source_folder_id = get_subfolder_id(drive, training_folder_id, '[원천]자동차번호판OCR데이터')

In [69]:
    print("라벨 파일 목록(처음 5개):", list(label_dict.items())[:5])
    print("이미지 파일 목록(처음 5개):", list(image_dict.items())[:5])

라벨 파일 목록(처음 5개): [('92라0979.json', '1foIJ3kuyu71y1BKPqFGng8sScpjCdP_4'), ('92도5577-60.json', '1fHtCmv9PRULg-NmhyprvcXML1k55o-aK'), ('92라0437.json', '1frVzgoW8BT_28Py_iDSw6EwWHkAoxIo_'), ('92마2666-2.json', '1gf72SG4QC2NGHQq_XLw0eBjAAbQPnAoI'), ('92도5577-6.json', '1fMZAobELiOLcPKmoozRZed261tlPl38G')]
이미지 파일 목록(처음 5개): [('경기71아3609-5.jpg', '1erSY3aC27uRCByer8SWnOFfjiz9ii7Lh'), ('경기71아1446-6.jpg', '1xGm-7bE40X4iD82IPa26iQNtWo29myzO'), ('경기71아1444-7.jpg', '1w05HzSSLn2YPtR54nWqY2Bza-tvMYJGC'), ('경기37바6657-2.jpg', '13VT8Re67GTSuXWdPONjU7EDGk7V8TY9e'), ('경기37바6117-6.jpg', '1mNXJWmJ41wMweHFnG6xmswMz7RbnZCsl')]


## root에서 실행하니 폴더만 3개 나옴

In [ ]:
# import pickle

# # 파일 목록 가져오기 및 캐싱
# def get_all_files(drive, folder_id):
#     if os.path.exists('file_list.pkl'):
#         with open('file_list.pkl', 'rb') as f:
#             all_files = pickle.load(f)
#     else:
#         # f"'{parent_id}' in parents and title = '{folder_title}' and mimeType = 'application/vnd.google-apps.folder' and trashed=false"
#         # query = f"'{folder_id}' in parents and trashed=false"
#         all_files = []
#         page_token = None
#         while True:
#             param = {'q':  f"'{folder_id}' in parents and trashed=false", 'maxResults': 1000}
#             if page_token:
#                 param['pageToken'] = page_token
#             files = drive.ListFile(param).GetList()
#             all_files.extend(files)
#             # if len(files) < 1000:
#             #     break
#         with open('file_list.pkl', 'wb') as f:
#             pickle.dump(all_files, f)
#     return all_files

# # 파일 목록 가져오기
# # 지정된 폴더 ID '10NnVsnoJuH4Ok56_TT5op-OEQrDY-2nZ' == Training에서 파일 목록 가져오기
# # 1jYZlhZJW94-34-DKpALvsauuoKL4s_Yh : Traing - 원천 자동차 번호판 데이터(jpg)
# folder_id = '10NnVsnoJuH4Ok56_TT5op-OEQrDY-2nZ'
# all_files = get_all_files(drive, folder_id)
# print(f"총 파일 수: {len(all_files)}")

# # 이미지 및 라벨 파일 분류
# image_files = {}
# label_files = {}

# for file in all_files:
#     if file['mimeType'] != 'application/vnd.google-apps.folder':
#         if file['title'].endswith('.jpg'):
#             image_files[file['title']] = file['id']
#         elif file['title'].endswith('.json'):
#             label_files[file['title']] = file['id']

# print(f"이미지 파일 수: {len(image_files)}")
# print(f"라벨 파일 수: {len(label_files)}")

In [41]:
import io
# 파일을 메모리로 읽어오기
def read_file_from_drive(drive, file_id):
    file = drive.CreateFile({'id': file_id})
    file_content = io.BytesIO(file.GetContentBinary())
    return file_content
    
# 예시: 첫 번째 이미지와 라벨 파일 읽어오기
# 파일 이름 매칭을 위해 공통된 부분을 사용한다고 가정
for image_name in image_dict:
    # 이미지 파일 읽기
    image_content = read_file_from_drive(drive, image_dict[image_name])
    np_image = np.frombuffer(image_content.getvalue(), np.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)

    # 대응되는 라벨 파일 이름 생성 (예: 이미지 파일명에서 확장자를 변경)
    label_name = image_name.replace('.jpg', '.json')

    if label_name in label_dict:
        # 라벨 파일 읽기
        label_content = read_file_from_drive(drive, label_dict[label_name])
        label_json = json.loads(label_content.getvalue().decode('utf-8'))

        # 이미지와 라벨을 사용하여 데이터셋에 추가하거나 모델에 입력
        # ...
    else:
        print(f"라벨 파일을 찾을 수 없습니다: {label_name}")

AttributeError: 'GoogleDriveFile' object has no attribute 'GetContentBinary'

## 데이터셋을 구성하는 방법

In [30]:
# 2. CustomDataset 클래스를 사용하는 방식
# 모델 학습을 위한 데이터로더 생성
import cv2
import json
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from PIL import Image  # OpenCV 이미지 -> PIL 이미지 변환에 필요

class CustomDataset(Dataset):
    def __init__(self, drive, image_dict, label_dict, transform=None):
        self.drive = drive
        self.image_dict = image_dict
        self.label_dict = label_dict
        self.transform = transform
        self.image_names = list(image_dict.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_id = self.image_dict[image_name]

        print(f"처리 중인 이미지 : {image_name}") # 이미지 이름 출력

        # # 이미지 읽기
        # image_content = read_file_from_drive(self.drive, image_id)
        # np_image = np.frombuffer(image_content.getvalue(), np.uint8)
        # img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
        
        # 이미지 파일 읽기
        file = self.drive.CreateFile({'id': image_id})
        file.GetContentFile('temp_image.jpg')  # 임시 파일로 저장
        img = cv2.imread('temp_image.jpg')  # OpenCV로 이미지 읽기

        # OpenCV 이미지를 PIL 이미지로 변환 (transforms 사용을 위해)
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        
        # 이미지 전처리
        if self.transform:
            img = self.transform(img)

        # 라벨 파일 읽기
        label_name = image_name.replace('.jpg', '.json')
        if label_name in self.label_dict:
            label_id = self.label_dict[label_name]
            label_file = self.drive.CreateFile({'id': label_id})
            label_file.GetContentFile('temp_label.json')  # 임시 파일로 저장

            # 파일이 비어 있는지 확인
            if os.path.getsize('temp_label.json') ==0:
                print(f"라벨 파일이 비어 있습니다: {label_name}")
                return img, None


            # JSON 파일 파싱
            try:            
                with open('temp_label.json', 'r', encoding='utf-8') as f:
                    label_json = json.load(f)
            except json.JSONDecodeError:
                print(f"JSONDecodeError: 라벨 파일이 유효하지 않습니다: {label_name}")
                return img, None

            # JSON에서 필요한 데이터 추출
            image_path = label_json.get('imagePath')
            value = label_json.get('value')
            label_id = label_json.get('id')

            # target에 필요한 정보 저장
            target = {'imagePath': image_path, 'value': value, 'id': label_id}
            # 바운딩 박스와 클래스 ID를 텐서로 변환
            # 필요한 형식에 맞게 변환하세요
            # 예를 들어, bbox 좌표를 정규화하거나 텐서로 변환
            # ...
        else:
            print(f"라벨 파일을 찾을 수 없습니다: {label_name}")
            target = None

        return img, target



## 데이터셋 및 데이터로더 생성 , 모델 학습 루프

In [31]:
from ultralytics import YOLO  # YOLOv8 모델 가져오기
# YOLOv8 모델 로드
model = YOLO('yolov8n.pt') 

In [32]:
import time
start_time = time.time()

# YOLOv8 모델을 훈련 모드로 전환
model.train()

# 전처리 정의 (리사이즈 후 텐서로 변환)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 모든 이미지를 224x224 크기로 리사이즈
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

# 데이터셋 및 데이터로더 생성
dataset = CustomDataset(drive, image_dict, label_dict, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0) # num_workers=0 : Drive와의 데이터 통신 문제 줄이기 위함 -> 단일 프로세스로 데이터 로딩 수행
num_epochs = 1

# 모델 학습 루프
for epoch in range(num_epochs):
    for imgs, targets in dataloader:
        # imgs와 targets를 모델에 입력
        # imgs: 이미지 텐서
        # targets: 바운딩 박스와 클래스 ID 등

        # 모델에 맞게 데이터 형식 조정
        # ...

        # 모델 출력 및 손실 계산
        outputs = model(imgs, targets)
        loss = outputs['loss'] # 모델이 예측한 결과로부터 손실 계산
        # loss = loss_function(outputs, targets)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")

처리 중인 이미지 : 전북73바1507-32.jpg
처리 중인 이미지 : 395가8909.jpg
처리 중인 이미지 : 서울31자5624.jpg
처리 중인 이미지 : 62두0185-3.jpg
처리 중인 이미지 : 01가5295-2.jpg
처리 중인 이미지 : 79누4310.jpg
처리 중인 이미지 : 경기37바4220-9.jpg
처리 중인 이미지 : 79거1580.jpg
처리 중인 이미지 : 18버9095-28.jpg
처리 중인 이미지 : 15머3508-3.jpg
처리 중인 이미지 : 128호2500.jpg
처리 중인 이미지 : 경기71아1306.jpg
처리 중인 이미지 : 경기37바1757-3.jpg
처리 중인 이미지 : 18보4596.jpg
처리 중인 이미지 : 52러9850.jpg
처리 중인 이미지 : 서울38아4922-2.jpg

0: 224x224 (no detections), 0.6ms
1: 224x224 1 keyboard, 0.6ms
2: 224x224 (no detections), 0.6ms
3: 224x224 1 cell phone, 0.6ms
4: 224x224 (no detections), 0.6ms
5: 224x224 (no detections), 0.6ms
6: 224x224 (no detections), 0.6ms
7: 224x224 (no detections), 0.6ms
8: 224x224 (no detections), 0.6ms
9: 224x224 (no detections), 0.6ms
10: 224x224 1 train, 0.6ms
11: 224x224 (no detections), 0.6ms
12: 224x224 1 parking meter, 0.6ms
13: 224x224 (no detections), 0.6ms
14: 224x224 (no detections), 0.6ms
15: 224x224 1 bus, 0.6ms
Speed: 0.2ms preprocess, 0.6ms inference, 1.8ms postprocess

AttributeError: 'list' object has no attribute 'loss'

## 2번째 방법으로 인증 관리 -> 기각

In [ ]:
#  Google 공식 인증 라이브러리를 사용하여 인증 관리
import requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# OAuth 2.0 클라이언트 ID를 사용하여 인증
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# OAuth 2.0 인증 설정
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # flow = InstalledAppFlow.from_client_secrets_file(
            #     'client_secrets.json', SCOPES)
            # creds = flow.run_local_server(port=0)
            auth_url, _ = flow.authorization_url(prompt='consent')
            print(f'Please go to this URL: {auth_url}')

            # 사용자가 브라우저에서 인증 후, 제공된 코드를 콘솔에 입력하도록 설정
            code = input('Enter the authorization code: ')
            creds = flow.fetch_token(code=code)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Google Drive API 호출
def list_files(creds):
    url = 'https://www.googleapis.com/drive/v3/files'
    headers = {'Authorization': f'Bearer {creds.token}'}
    response = requests.get(url, headers=headers)
    return response.json()

creds = authenticate()
files = list_files(creds)
print(files)


## 현재 실행 환경 확인 방법

In [21]:
# 로컬인지 서버인지 확인
# !hostname # jupyter02

# GPU 사용 여부
# print(torch.cuda.is_available()) # True

# 사용 중인 GPU 정보 확인
# !nvidia-smi

0
Tesla V100-PCIE-32GB


In [22]:
# pip와 python 경로 확인
import sys
print("Python 경로:", sys.executable)

import subprocess
pip_path = subprocess.check_output(['which', 'pip']).decode('utf-8').strip()
print("pip 경로:", pip_path)

# sys.executable을 통해 나의 python 경로에 tqdm 설치하기
import sys
!{sys.executable} -m pip install tqdm

Python 경로: /home/j-j11a303/.conda/envs/urecar/bin/python
pip 경로: /opt/tljh/user/bin/pip


## 기각되었던 코드들 , 필요한게 있을 수 있음

In [39]:
# import os
# folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# for file in file_list[:10]:
#     print(f"Downloading {file['title']} from {file['id']}")

In [ ]:
# # 1. dataset 리스트를 사용하는 방식

# # 이미지와 라벨을 매핑하여 데이터셋 구성
# # 데이터셋 리스트 생성
# dataset = []

# for image_name in image_dict:
#     # 이미지 파일 읽기
#     image_content = read_file_from_drive(drive, image_dict[image_name])
#     np_image = np.frombuffer(image_content.getvalue(), np.uint8)
#     img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)

#     # 대응되는 라벨 파일 이름 생성
#     label_name = image_name.replace('.jpg', '.json')

#     if label_name in label_dict:
#         # 라벨 파일 읽기
#         label_content = read_file_from_drive(drive, label_dict[label_name])
#         label_json = json.loads(label_content.getvalue().decode('utf-8'))

#         # 필요한 데이터 추출 (예: 바운딩 박스, 클래스 등)
#         # 라벨 형식에 따라 아래 부분을 수정하세요
#         annotations = []
#         for obj in label_json['objects']:
#             bbox = obj['bounding_box']
#             class_id = obj['class_id']
#             annotations.append({'bbox': bbox, 'class_id': class_id})

#         # 데이터셋에 추가
#         dataset.append({'image': img, 'annotations': annotations})
#     else:
#         print(f"라벨 파일을 찾을 수 없습니다: {label_name}")


In [13]:
# 파일을 다운 받지않고 바로 메모리에서 처리하기!!!!!!!!! -> io.BytesIO 사용하기
# Google Drive에서 이미지와 라벨 파일을 다운로드하여 로컬 경로로 저장
def download_drive_folder(folder_id, local_path):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    if not os.path.exists(local_path):
        os.makedirs(local_path)
    
    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 하위 폴더인 경우
            subfolder_local_path = os.path.join(local_path, file['title'])
            print(f"Entering folder: {file['title']}")
            download_drive_folder_recursive(file['id'], subfolder_local_path)  # 재귀적으로 하위 폴더 처리
        else:
            print(f"Downloading {file['title']} from Google Drive")
            file.GetContentFile(os.path.join(local_path, file['title']))

# Google Drive에서 폴더 ID를 이용해 데이터 다운로드
image_folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'  # Google Drive의 이미지 폴더 ID 입력
label_folder_id = '17KDAGcw4JLD3COQHQVYk68AyxUysTGPA'  # Google Drive의 라벨 폴더 ID 입력

image_local_path = './images'
label_local_path = './labels'

# 이미지 및 라벨 다운로드
download_drive_folder(image_folder_id, image_local_path)
download_drive_folder(label_folder_id, label_local_path)

# LaneDataset 클래스 정의
class LaneDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_folders = os.listdir(image_dir)
        self.label_folders = os.listdir(label_dir)

        # 각 폴더 안에 있는 이미지 파일들을 미리 저장
        self.image_files = []
        for folder in self.image_folders:
            folder_path = os.path.join(self.image_dir, folder)
            images = os.listdir(folder_path)  # 이미지 파일 리스트
            for img_file in images:
                self.image_files.append((folder, img_file)) # (폴더명, 이미지 파일명) 저장

    def __len__(self):
        return len(self.image_files)  # 모든 이미지 파일의 총 개수

    def __getitem__(self, idx):
        # 이미지 파일 정보
        folder, img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, folder, img_file)

        # 해당 이미지 파일에 대응하는 라벨 폴더 및 라벨 파일
        label_folder_path = os.path.join(self.label_dir, folder)
        label_file = os.listdir(label_folder_path)[0]  # 라벨 파일 선택
        label_path = os.path.join(label_folder_path, label_file)

        # 이미지 로드
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 라벨 로드 (JSON 파일에서 차선 종류 및 위반 여부 정보 추출)
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # 위반 여부 및 차선 종류 추출
        violation, lane_type = self.extract_violation(label_path)

        # 필요한 경우 데이터 변환 적용
        if self.transform:
            image = self.transform(image)

        return image, violation, lane_type

    # JSON 파일에서 위반 여부와 차선 종류 추출 함수
    def extract_violation(self, json_file_path):
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # 위반 여부와 차선 종류 기본값 설정
        violation = False  # 기본적으로는 위반 아님
        lane_type = None   # 차선 종류는 None을 기본값으로 설정

        # "data_set_info"에서 "data" 항목에 있는 객체 정보 확인
        for obj in data['data_set_info']['data']:
            object_label = obj['value']['object_Label']

            # 위반 여부 판단
            if 'vehicle_attribute' in object_label and object_label['vehicle_attribute'] == 'violation':
                violation = True  # 위반 차량 있음
            # 차선 종류 판단
            if 'lane_type' in object_label:
                lane_type = object_label['lane_type']

        return violation, lane_type

# 데이터 변환 설정
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # ResNet 입력 크기와 맞추기
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지와 라벨 데이터셋 생성
dataset = LaneDataset(image_dir=image_local_path, label_dir=label_local_path, transform=data_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader에서 데이터 순차적으로 불러오기
for images, (violations, lane_types) in data_loader:
    print(images.shape, violations, lane_types)

Entering folder: [25627]A_BLUE


NameError: name 'download_drive_folder_recursive' is not defined

In [161]:
# !google auth oauthlib --version
!torch --version

/bin/bash: torch: command not found


In [75]:
!jupyter labextension install @jupyterlab/google-drive

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/opt/tljh/user/lib/python3.10/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for details: /tmp/jupyterlab-debug-5qghd27m.log
  warnings.warn(f"See the log file for details: {log

In [ ]:
# token 설정과 관련된 거 였는데, token 설정 잘 됨!
import requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# OAuth 2.0 클라이언트 ID를 사용하여 인증
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# OAuth 2.0 인증 설정
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# 최상위 폴더의 파일 및 폴더 목록을 가져오기 위한 함수
def list_root_files(creds, folder_id):
    url = f'https://www.googleapis.com/drive/v3/files?q=\'{folder_id}\' in parents'
    headers = {'Authorization': f'Bearer {creds.token}'}
    params = {
        'pageSize': 100,  # 가져올 최대 파일 수
        'fields': 'nextPageToken, files(id, name, mimeType)',  # 파일 이름, ID, MIME 타입
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# 폴더 ID 설정 (해당 폴더의 ID를 사용)
folder_id = '1nor9jn1J30mZN_168SsNj1ArO4duu1VT'  # Colab Notebooks/SpecializedProject/content/차로 위반 영상 데이터/sample_datas 
# 이 안에 라벨링데이터/TL/A/BLUE 안에 여러 폴더 + 원천데이터/TS1/A/BLUE 안에 여러 폴더

creds = authenticate()
files = list_root_files(creds, folder_id)
print('폴더 ID까지 설정 완료', files)
# 최상위 폴더에 있는 파일 및 폴더 출력
for file in files.get('files', []):
    print(f"Name: {file['name']}, ID: {file['id']}, MIME Type: {file['mimeType']}")


In [13]:
# 파일을 다운 받지않고 바로 메모리에서 처리하기
import io
import json
import requests
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
# import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import numpy as np

# OAuth 2.0 인증 설정 (기존 코드와 동일)
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Google Drive 파일을 메모리에서 읽는 함수
def download_drive_file(file_id, creds):
    url = f'https://www.googleapis.com/drive/v3/files/{file_id}?alt=media'
    headers = {'Authorization': f'Bearer {creds.token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return io.BytesIO(response.content)
    else:
        raise Exception(f"Error downloading file {file_id}, status code: {response.status_code}")

# 이미지와 라벨 데이터셋 정의 (메모리에서 처리)
class LaneDataset(Dataset):
    def __init__(self, image_file_ids, label_file_ids, creds, transform=None):
        self.image_file_ids = image_file_ids
        self.label_file_ids = label_file_ids
        self.transform = transform
        self.creds = creds

    def __len__(self):
        return len(self.image_file_ids)  # 모든 이미지 파일의 총 개수

    def __getitem__(self, idx):
        # Google Drive에서 이미지를 메모리로 다운로드
        img_file_id = self.image_file_ids[idx]
        img_data = download_drive_file(img_file_id, self.creds)
        image = np.frombuffer(img_data.read(), np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_GR2RGB)

        # Google Drive에서 라벨 데이터를 메모리로 다운로드
        label_file_id = self.label_file_ids[idx]
        label_data = download_drive_file(label_file_id, self.creds)
        label_json = json.load(label_data)

        # 위반 여부 및 차선 종류 추출
        violation, lane_type = self.extract_violation(label_json)

        # 필요한 경우 데이터 변환 적용
        if self.transform:
            image = self.transform(image)

        return image, violation, lane_type

    # JSON 파일에서 위반 여부와 차선 종류 추출 함수
    def extract_violation(self, label_data):
        violation = False  # 기본적으로는 위반 아님
        lane_type = None   # 차선 종류는 None을 기본값으로 설정

        for obj in label_data['data_set_info']['data']:
            object_label = obj['value']['object_Label']
            if 'vehicle_attribute' in object_label and object_label['vehicle_attribute'] == 'violation':
                violation = True
            if 'lane_type' in object_label:
                lane_type = object_label['lane_type']

        return violation, lane_type

# 데이터 변환 설정
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # ResNet 입력 크기와 맞추기
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Google Drive에서 가져올 이미지와 라벨 파일 ID 목록
image_file_ids = ['image_file_id_1', 'image_file_id_2']  # 실제 이미지 파일 ID 입력
label_file_ids = ['label_file_id_1', 'label_file_id_2']  # 실제 라벨 파일 ID 입력

creds = authenticate()

# 이미지와 라벨 데이터셋 생성
dataset = LaneDataset(image_file_ids=image_file_ids, label_file_ids=label_file_ids, creds=creds, transform=data_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader에서 데이터 순차적으로 불러오기
for images, (violations, lane_types) in data_loader:
    print(images.shape, violations, lane_types)


Exception: Error downloading file image_file_id_2, status code: 404

In [2]:
!pip show torch
!pip show torchvision

Name: torch
Version: 2.4.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/j-j11a303/.local/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, sympy, triton, typing-extensions
Required-by: torchaudio, torchvision
Name: torchvision
Version: 0.19.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /home/j-j11a303/.local/lib/python3.10/site-packages
Requires: numpy, pillow, torch
Required-by: 
